In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import os

In [2]:
# Add all NEW data files to one dataframe 
# Make sure all processed csv-s are in the processed folder

def create_big_df(filename):

    df_big = pd.DataFrame()
    df_data = pd.read_csv(filename)
    df_data.head()
    df_big = df_big.append(df_data)

In [3]:
def save_to_database(df_big):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')

    df_big.to_sql(name='daily_stats', con=engine, if_exists='append', index=False)

In [20]:
counter= 0

query_str = open('static/sql/world_days.sql')
query_text = ""

for text in query_str:
    query_text = query_text + text

counter = engine.execute(query_text)
print(counter)

for i in counter:
    print(i[0])

40


In [6]:
def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
    connection = engine.connect()
    connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
    connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [4]:
df_big = pd.DataFrame()

for file in os.listdir("static/data"):
    if file.endswith(".csv"):
        print("Now Processing: " + "static/data/" + file)
        date = file[3:13]
        filename = "static/data/" + file
        df_data = pd.read_csv(filename)
        df_big = df_big.append(df_data)
        continue
    else:
        continue
        
df_big.head()
df_big.to_csv("static/out/out.csv", encoding='utf-8')

Now Processing: static/data/df_2020-02-25.csv
Now Processing: static/data/df_2020-02-26.csv
Now Processing: static/data/df_2020-02-27.csv
Now Processing: static/data/df_2020-02-28.csv
Now Processing: static/data/df_2020-02-29.csv
Now Processing: static/data/df_2020-03-01.csv
Now Processing: static/data/df_2020-03-02.csv


In [5]:
df_big.count()
df_big.rename(columns = {'provinceName':'province_name', 
                         'provinceShortName':'province_short', 
                         'confirmedCount':'conf_count', 
                         'suspectedCount': 'susp_count', 
                         'curedCount':'cured_count',
                         'deadCount':'dead_count'}, inplace = True) 
save_to_database(df_big)

In [9]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_all.csv"
df_world = pd.read_csv(url_world)
df_world.head()
save_world_to_database(df_world)

In [6]:
query_str = open('static/sql/test_query.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query

select date, 
	sum(conf_count) confirmed, 
	sum(susp_count) suspected,
	sum(cured_count) cured,
	sum(dead_count) dead
from daily_stats 
group by date 
order by date


,date,confirmed,suspected,cured,dead
0,2020-01-26,1812,0,49,55
1,2020-01-27,2081,0,49,56
2,2020-01-28,2858,0,56,82
3,2020-01-29,4633,0,73,106
4,2020-01-30,6095,1,120,133
5,2020-01-31,8150,0,135,171
6,2020-02-01,11230,0,70,258
7,2020-02-02,13831,0,322,304
8,2020-02-03,14525,1,435,305
9,2020-02-04,19690,0,527,425


In [7]:
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query


select  
	country, 
	sum(conf_count) total,
	sum(conf_count)-sum(cured_count)-sum(dead_count) sick, 
	sum(cured_count) cured, 
	sum(dead_count) dead,
	max(date)
    from daily_stats_world 
    where date = (select max(date ) from daily_stats_world) 
    group by country 
    order by total desc 
	limit 10


,country,total,sick,cured,dead,max
0,China,79826,34838,42118,2870,2020-03-01
1,South Korea,3736,3689,30,17,2020-03-01
2,Italy,1694,1577,83,34,2020-03-01
3,Iran,978,749,175,54,2020-03-01
4,Others,705,689,10,6,2020-03-01
5,Japan,256,218,32,6,2020-03-01
6,France,130,116,12,2,2020-03-01
7,Germany,130,114,16,0,2020-03-01
8,Singapore,106,34,72,0,2020-03-01
9,Hong Kong,96,58,36,2,2020-03-01
